In [13]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch

In [14]:
!pwd

/bi/ala/scratch/pmehrbod/GAN


In [7]:
label_numeric={folder:i+1 for i,folder in enumerate(os.listdir("selection1"))}
    

In [15]:
meta_data=[]
for folder in os.listdir("selection1"):
    for filename in os.listdir(f"selection1/{folder}"):
        if filename.endswith(".jpg"):
            meta_data.append({"name":filename,"label":folder,"path":f"{folder}/{filename}"})

In [16]:
meta_data_df=pd.DataFrame(meta_data)
meta_data_df.to_csv("selection1_meta_data.csv")

In [10]:
meta_data_df.groupby("label").count()

,name,path
label,,
Abstract,241,241
Abstract_Blossum,231,231
Almsost_Ditsy,371,371
Big_Flowers,197,197
Botanical,883,883
Checks,217,217
Chevron,161,161
Ditsy,681,681
Leopard,608,608


In [11]:
np.array(Image.open("selection1/Abstract/101207048-tropical-i.jpg").convert('RGB')).shape

(266, 266, 3)

In [12]:

torch.manual_seed(0) 
class FabricDataset(Dataset):
    
    def __init__(self, data_folder, meta):
        super().__init__()
        self.df=pd.read_csv(meta, index_col=0)
        paths=self.df.path.tolist()
        labels=[label_numeric[l] for l in self.df.label.tolist()]
        
        self.data = [(f"{data_folder}/{path}", label) for path, label in zip(paths, labels)]

    def __getitem__(self, index):
        inst = self.data[index]
        return self.imge_to_tensor(inst[0]), inst[1]

    def __len__(self):
        return len(self.data)
    
    def imge_to_tensor(self, f):
        img = Image.open(f).convert('RGB')
        # trans_compose=transforms.ToTensor()
        transform = transforms.Compose([transforms.Resize([200, 200]),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.5], [0.5])])
        img_tensor=transform(img)
        return img_tensor

In [43]:
dataset = FabricDataset("selection1","selection1_meta_data.csv")
len(dataset)

6235

In [44]:
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)

In [45]:
BATCH_SIZE = 128

In [46]:
# for checking data, lets create iterator
# We won't use this iterator for training purpose
dataiter = iter(dataloader)
images, labels = next(dataiter)
# MNIST image size is 1*28*28
img_size = images.shape[2]
print(images.shape)
# Create images grid of 16 rows, batch size need to be multiple of 16
# for batch size of 128, 128 images will be arranged in 8*16 grid 
torchvision.utils.save_image(images, f"{DATASET_NAME}_input_grid.jpg", nrow=16, padding=0, normalize=True)

NameError: Caught NameError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/bi/home/pmehrbod/.conda/envs/GAN/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/bi/home/pmehrbod/.conda/envs/GAN/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/bi/home/pmehrbod/.conda/envs/GAN/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_41492/1993956779.py", line 14, in __getitem__
    return self.imge_to_tensor(inst[0]), inst[1]
  File "/tmp/ipykernel_41492/1993956779.py", line 22, in imge_to_tensor
    transform = transforms.Compose([transforms.Resize([200, 200]),
NameError: name 'transforms' is not defined
